<a href="https://colab.research.google.com/github/guidosassaroli/guidosassaroli.github.io/blob/master/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example RL with Cart Pole 

In [1]:
#Install Dependencies 

!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: gast
    Found existing installa

In [28]:
import gym 

In [29]:
import random 

In [32]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [33]:
actions

2

In [34]:
pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
episodes = 10
for episode in range(1, episodes+1):
  state = env.reset()
  done = False
  score = 0 
  
  while not done:
      #env.render()
      action = random.choice([0,1])
      n_state, reward, done, info = env.step(action)
      score+=reward
  print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:13.0
Episode:2 Score:27.0
Episode:3 Score:11.0
Episode:4 Score:31.0
Episode:5 Score:16.0
Episode:6 Score:61.0
Episode:7 Score:20.0
Episode:8 Score:23.0
Episode:9 Score:34.0
Episode:10 Score:11.0


Deep Learning Model with Keras

In [36]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [37]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [38]:
model = build_model(states, actions)

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


Build agent with Keras-RL

In [40]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [41]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn



In [42]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training_utils.py:1446: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:02 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   38/10000 [..............................] - ETA: 2:35 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

10000/10000 [==============================] - 62s 6ms/step - reward: 1.0000
94 episodes - episode_reward: 102.223 [9.000, 491.000] - loss: 1.928 - mae: 19.789 - mean_q: 40.131

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: 1.0000
43 episodes - episode_reward: 239.814 [158.000, 394.000] - loss: 2.302 - mae: 41.520 - mean_q: 83.938

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: 1.0000
40 episodes - episode_reward: 245.700 [147.000, 344.000] - loss: 1.469 - mae: 43.896 - mean_q: 88.519

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 1.0000
39 episodes - episode_reward: 260.462 [147.000, 500.000] - loss: 1.049 - mae: 42.603 - mean_q: 85.738

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 1.0000
done, took 318.952 seconds


In [43]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 395.000, steps: 395
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 413.000, steps: 413
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 206.000, steps: 206
Episode 6: reward: 391.000, steps: 391
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 164.000, steps: 164
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 274.000, steps: 274
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 370.000, steps: 370
Episode 15: reward: 449.000, steps: 449
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 467.000, steps: 467
Episode 19: reward: 387.000, steps: 387
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 373.000, steps: 373
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 478.000, steps: 478
Episode 24: reward: 343.000, steps: 343
Episode 25: reward: 

In [50]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


Reloading Agent from memort

In [45]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [46]:


del model
del dqn
del env



In [47]:


env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])



In [48]:
dqn.load_weights('dqn_weights.h5f')

In [49]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
